In [ ]:
import polars as pl
import duckdb
from pathlib import Path
import os
import tabulate

In [ ]:
def get_bucket_stats(bucket_dir: Path) -> tuple[int, float, list[Path]]:
    files = sorted(bucket_dir.glob("bucket_*.parquet"))
    total_rows = 0
    total_size_mb = 0.0
    missing = []
    for f in files:
        if not f.exists():
            missing.append(f)
            continue
        try:
            df = pl.read_parquet(f)
            total_rows += df.shape[0]
            total_size_mb += f.stat().st_size / (1024 * 1024)
        except Exception:
            missing.append(f)
    return total_rows, round(total_size_mb, 2), missing

def get_manifest_stats(manifest_db: Path, cenyear: int) -> tuple[int, list[str]]:
    with duckdb.connect(str(manifest_db)) as con:
        df = pl.from_arrow(con.execute(
            "SELECT file, record_count FROM manifest WHERE cenyear = ?", [cenyear]
        ).arrow())
    total = df["record_count"].sum()
    files = df["file"].to_list()
    return total, files

def get_index_stats(index_db: Path, cenyear: int) -> int:
    year_str = str(cenyear)
    with duckdb.connect(str(index_db)) as con:
        return con.execute("SELECT COUNT(*) FROM index WHERE hhid LIKE ?", [f"{year_str}%"]).fetchone()[0]

In [ ]:
def summarize_all_years(basepath: Path, folder_map: dict[str, Path], output_csv: Path):
    manifest_db = basepath / "manifests" / "manifest.duckdb"
    index_db = basepath / "index" / "search_index.duckdb"

    summary_rows = []
    for year_str, bucket_dir in folder_map.items():
        cenyear = int(year_str)
        bucket_rows, bucket_size, missing_files = get_bucket_stats(bucket_dir)
        manifest_rows, manifest_files = get_manifest_stats(manifest_db, cenyear)
        index_rows = get_index_stats(index_db, cenyear)

        expected_bucket_count = len(manifest_files)
        actual_bucket_count = len(bucket_dir.glob("bucket_*.parquet"))

        problems = []
        if len(missing_files) > 0:
            problems.append("MissingFiles")
        if actual_bucket_count != expected_bucket_count:
            problems.append("MissingBuckets")
        if bucket_rows != manifest_rows:
            problems.append("RowMismatch")
        if index_rows == 0:
            problems.append("IndexEmpty")
        elif index_rows < manifest_rows * 0.5:
            problems.append("IndexMismatch")

        summary_rows.append({
            "cenyear": cenyear,
            "bucket_rows": bucket_rows,
            "bucket_size_mb": bucket_size,
            "manifest_rows": manifest_rows,
            "index_rows": index_rows,
            "missing_files": len(missing_files),
            "status": "OK" if not problems else ",".join(problems)
        })

    df_summary = pl.DataFrame(summary_rows)

    # Print tabulated summary
    print("\n📊 Summary Table:")
    print(tabulate(df_summary.to_dicts(), headers="keys", tablefmt="github"))

    # Save to CSV
    df_summary.write_csv(output_csv)
    print(f"\n✅ Summary written to {output_csv}")



In [ ]:
def create_locations_table(basepath: Path):
    loc_folder = basepath / "locations"
    loc_csv = loc_folder / "cloc.csv"
    loc_db = loc_folder / "locations.duckdb"

    df = pl.read_csv(loc_csv).with_row_count(name="locid")
    df = df.select(["cloc", "stateicp", "countyicp", "statename", "countyname", "locid"])

    loc_folder.mkdir(exist_ok=True)
    with duckdb.connect(str(loc_db)) as con:
        con.execute("CREATE OR REPLACE TABLE locations AS SELECT * FROM df")

    print(f"✅ Created locations.duckdb with {df.shape[0]} rows.")

In [ ]:
from tabulate import tabulate

def verify_cloc_values(basepath: Path, folder_map: dict[str, Path]):
    loc_db = basepath / "locations" / "locations.duckdb"
    manifest_db = basepath / "manifests" / "manifest.duckdb"
    index_db = basepath / "index" / "search_index.duckdb"

    with duckdb.connect(str(loc_db)) as con:
        cloc_known = set(con.execute("SELECT cloc FROM locations").fetchall())

    cloc_used = set()

    # From manifest
    with duckdb.connect(str(manifest_db)) as con:
        rows = con.execute("""
            SELECT DISTINCT min_locid AS cloc FROM manifest
            UNION
            SELECT DISTINCT max_locid AS cloc FROM manifest
        """).fetchall()
        cloc_used.update([r[0] for r in rows if r[0] is not None])

    # From index
    with duckdb.connect(str(index_db)) as con:
        rows = con.execute("SELECT DISTINCT cloc FROM index").fetchall()
        cloc_used.update([r[0] for r in rows if r[0] is not None])

    # From buckets
    for year_str, bucket_dir in folder_map.items():
        for f in bucket_dir.glob("bucket_*.parquet"):
            try:
                df = pl.read_parquet(f)
                if "cloc" in df.columns:
                    cloc_used.update(df["cloc"].unique().to_list())
            except Exception:
                print(f"⚠️ Failed to read {f}")

    missing = sorted(cloc_used - cloc_known)
    if missing:
        print(f"\n⚠️ {len(missing)} cloc values used in data but missing from locations table:")
        print(tabulate([{"cloc": c} for c in missing], headers="keys", tablefmt="github"))
    else:
        print("✅ All cloc values used in the database are present in the locations table.")

In [ ]:
def missing_cloc_values(basepath: Path, folder_map: dict[str, Path]):
    missing_files = []

    for year_str, bucket_dir in folder_map.items():
        for f in bucket_dir.glob("bucket_*.parquet"):
            try:
                df = pl.read_parquet(f, n_rows=1)
                if "cloc" not in df.columns:
                    missing_files.append({"year": year_str, "file": f.name})
            except Exception:
                missing_files.append({"year": year_str, "file": f.name + " (unreadable)"})

    if missing_files:
        print(f"\n⚠️ {len(missing_files)} bucket files missing 'cloc' column:")
        print(tabulate(missing_files, headers="keys", tablefmt="github"))
    else:
        print("✅ All bucket files contain a 'cloc' column.")

In [ ]:
def create_search_index_indexes(basepath: Path):
    index_db = basepath / "index" / "search_index.duckdb"
    with duckdb.connect(str(index_db)) as con:
        con.execute("CREATE INDEX IF NOT EXISTS idx_hhid ON index(hhid);")
        con.execute("CREATE INDEX IF NOT EXISTS idx_cloc ON index(cloc);")
    print("✅ Created indexes on search_index: hik (unique), hhid (filterable), cloc (filterable)")

In [ ]:
def create_manifest_indexes(basepath: Path):
    manifest_db = basepath / "manifests" / "manifest.duckdb"
    with duckdb.connect(str(manifest_db)) as con:
        # Index for fast filtering by year
        con.execute("CREATE INDEX IF NOT EXISTS idx_manifest_cenyear ON manifest(cenyear);")

        # Index for range-based hhid filtering (used in BETWEEN queries)
        con.execute("CREATE INDEX IF NOT EXISTS idx_manifest_min_hhid ON manifest(min_hhid);")
        con.execute("CREATE INDEX IF NOT EXISTS idx_manifest_max_hhid ON manifest(max_hhid);")

        # Optional: compound index for year + hhid range filtering
        con.execute("CREATE INDEX IF NOT EXISTS idx_manifest_year_hhid ON manifest(cenyear, min_hhid, max_hhid);")

    print("✅ Created indexes on manifest: cenyear, min_hhid, max_hhid, and compound year+hhid")